## setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import library
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

# file path
path = '/content/drive/MyDrive/python/공공데이터 공모전/data/'

## 1. 기업 DSC(import) & HS10 코드 임베딩 맵핑


---

코사인 유사도를 기반으로 거리가 가까운 맵핑 상위 30개를 추출


In [ ]:
data = pd.read_excel(path + '비식별된 해외기업별 영문 텍스트데이터.xlsx').drop_duplicates('DSC').reset_index(drop=True)
data_tr = pd.read_json(path+'data(import)_transformer_embedding.json')
hs_tr = pd.read_json(path+'hs_transformer_embedding.json')

### embedding mapping

In [ ]:
data_tr['matched'] = '0'
data_tr['maxscore'] = 0

In [ ]:
for i in range(en(data_tr)):
  emb = data_tr['transformer_embedding'][i]
  tmp = hs_tr.copy()
  tmp['cos_sim'] = tmp['transformer_embedding'].apply(lambda x : cosine_similarity([x], [emb])[0][0])
  tmp = tmp.sort_values(by=["cos_sim"], ascending=[False])
  data_tr['maxscore'][i] = np.max(tmp['cos_sim'])
  match_dict = {}
  for _, (hslist, score) in enumerate(zip(list(tmp['HS부호'])[:30],list(tmp['cos_sim'])[:30])):
    match_dict[score] = hslist
  data_tr['matched'][i] = match_dict

In [ ]:
data_tr = data_tr[['DSC', 'DSC(import)', 'LLMimport', 'transformer_embedding', 'matched', 'maxscore']]

In [ ]:
data_tr.to_json(path+'mapping30.json')

## 2. 기업 DSC의 예외사항(except, other than) & HS10 코드 임베딩 맵핑


---


코사인 유사도를 기반으로 거리가 가까운 맵핑 상위 30개를 추출


In [ ]:
ex_tr = pd.read_json(path+'ex_other_transformer_embedding.json')
hs_tr = pd.read_json(path+'hs_transformer_embedding.json')

### embedding mapping

In [ ]:
# except
ex_tr['ex_matched'] = '0'
ex_tr['ex_maxscore'] = 0
# other than
ex_tr['ot_matched'] = '0'
ex_tr['ot_maxscore'] = 0

In [ ]:
# except 구문 HS10 코드와 맵핑
for i in list(ex_tr[ex_tr['except']!='0'].index):
  emb = ex_tr['except_embedding'][i]
  tmp = hs_tr.copy()
  tmp['cos_sim'] = tmp['transformer_embedding'].apply(lambda x : cosine_similarity([x], [emb])[0][0])
  tmp = tmp.sort_values(by=["cos_sim"], ascending=[False])
  ex_tr['ex_maxscore'][i] = np.max(tmp['cos_sim'])
  match_dict = {}
  for _, (hslist, score) in enumerate(zip(list(tmp['HS부호'])[:30],list(tmp['cos_sim'])[:30])):
    match_dict[score] = hslist
  ex_tr['ex_matched'][i] = match_dict

In [ ]:
# other than 구문 HS10 코드와 맵핑
for i in list(ex_tr[ex_tr['other']!='0'].index):
  emb = ex_tr['other_embedding'][i]
  tmp = hs_tr.copy()
  tmp['cos_sim'] = tmp['transformer_embedding'].apply(lambda x : cosine_similarity([x], [emb])[0][0])
  tmp = tmp.sort_values(by=["cos_sim"], ascending=[False])
  ex_tr['ot_maxscore'][i] = np.max(tmp['cos_sim'])
  match_dict = {}
  for _, (hslist, score) in enumerate(zip(list(tmp['HS부호'])[:30],list(tmp['cos_sim'])[:30])):
    match_dict[score] = hslist
  ex_tr['ot_matched'][i] = match_dict

In [ ]:
ex_tr.to_json(path+'ex_other_mapping.json')